In [93]:
from openai import OpenAI
import os
client = OpenAI(api_key=os.getenv('Q_API_KEY'))

# Função para gerar mensagens
def generate_message(question, answer_text):
    return [
        {
            "role": "system",
            "content": f"Você é um professor criterioso de {question['disciplineName']} corrigindo uma questão discursiva de uma prova e que busca ajudar o seu aluno a melhorar.",
        },
        {
            "role": "user",
            "content": (
                f"Com base em provas e concursos passados, simule uma questão discursiva de concurso público (pode ser real) sobre {question['disciplineName']}\n"
                f"- Crie uma resposta ideal de exemplo\n"
                f"- Crie instruções para a correção da pergunta\n"
                f"- Coloque a pergunta dentro da tag <question>, o exemplo de resposta dentro da tag <example>"
            ),
        },
        {
            "role": "assistant",
            "content": (
                f"Pergunta: <question>{question['statement']}</question>"
            ),
        },
        {
            "role": "user",
            "content": (
                "Com base nas instruções discutidas e nos itens listados abaixo, preciso que você avalie a minha resposta como um professor.\n"
                "- Verifique se a resposta é ofensiva ou irrelevante em relação à pergunta. Caso seja ofensiva ou irrelevante, responda com algum texto como o exemplo \"Não foi possível avaliar sua resposta, pois não está relacionada com a pergunta proposta\" e não continue a avaliação.\n"
                "- Se a minha resposta for a própria pergunta ou muito parecida, responda com algum texto parecido com o exemplo \"Não é possível avaliar a sua resposta, pois você simplesmente repetiu a pergunta original. Lembre-se de que em uma questão discursiva, é fundamental desenvolver uma argumentação própria e não apenas repetir o enunciado.\" e não continue a avaliação.\n"
                "- Seja gentil e discorra de forma clara e educativa os pontos e fale na primeira pessoa como meu professor.\n"
                "- Considere a minha resposta somente o conteúdo de <userAnswer> que virá ao final das instruções.\n"
                "- Separe sua avaliação em 3 partes:\n"
                "  -- 1) Conceito Geral (Nessa parte, avalie se todos os conceitos e informações esperadas para a resposta foram abordados, use o conteúdo de <example> como comparação, mas não se limite a ele).\n"
                "  -- 2) Coerência e Coesão (Nessa parte, verifique a coerência e coesão da minha resposta, como os conceitos foram apresentados e o que pode melhorar).\n"
                "  -- 3) Dicas (Nessa parte, me dê dicas de como melhorar a minha resposta, quais conceitos deveria ter abordado, como poderia ter redigido de uma forma melhor e outros pontos que considerar relevante.)\n"
                "- Não utilize tags HTML.\n"
                "- Não utilize de tags Markdown como **.\n"
                f"Minha resposta: <userAnswer>{answer_text}</userAnswer>"
            ),
        }
    ]

# Função principal para gerar feedback
def generate_grade_stream(
    question,
    content,
    use_strategy = "default",
):

    # Configuração da estratégia
    # gpt-3.5-turbo
    # gpt-4o-mini
    strategies = {
        "default": {
            "fn": generate_message,
            "model": "gpt-3.5-turbo"
        }
    }

    strategy = strategies[use_strategy]
    messages = strategy["fn"](question, content)
    model = strategy["model"]

    # Gerar resposta da API
    response = client.chat.completions.create(
        model=model,
        temperature=0.75,
        top_p=0.75,
        max_tokens=1500,
        messages=messages,
    )

    # Retornar resposta
    feedback = response.choices[0].message

    print(feedback.content)

question = {
    "disciplineName": "Português",
    "statement": "A frase “Aos políticos, cabe as representações dos nichos que representa.” está correta gramaticalmente?"
}

# Variável representando a resposta do usuário
answer = "A frase não está correta gramaticalmente, pois o objeto indireto “Aos políticos” não pode ser separado por vírgula."

generate_grade_stream(question, answer)


Conceito Geral: Sua resposta abordou corretamente a questão gramatical presente na frase fornecida, apontando o erro de pontuação ao separar o objeto indireto "Aos políticos" por vírgula. Além disso, você identificou que a frase está incorreta gramaticalmente, o que demonstra compreensão do problema apresentado.

Coerência e Coesão: Sua resposta é concisa e direta, focando no ponto gramatical que precisa ser corrigido na frase. No entanto, seria interessante expandir um pouco mais sua explicação, mencionando a função correta da preposição "a" e do objeto indireto na construção da frase. Além disso, poderia relacionar o uso correto da pontuação com a clareza e correção do texto.

Dicas: Para melhorar sua resposta, sugiro que você amplie a explicação, contextualizando o uso correto da preposição "a" e a importância de manter a coesão e clareza na escrita ao evitar erros de pontuação. Também seria válido exemplificar como a correção da frase poderia ser feita, demonstrando a aplicação cor